### Fetching Json from API

In [1]:
import requests
import json
import sqlite3

# Step 1: Fetch JSON data
# prize
response1 = requests.get("https://api.nobelprize.org/v1/prize.json")
data1 = response1.json()
# laureate
response2 = requests.get("https://api.nobelprize.org/v1/laureate.json")
data2 = response2.json()

In [2]:
# Step 2: Parse JSON data
prize_data = data1['prizes']
laureate_data = data2['laureates']

In [3]:
# print(prize_data)

In [4]:
# print(laureate_data)

In [5]:
# Read the JSON file
with open('prize_data.json') as f:
    prize_data = json.load(f)
# Read the JSON file
with open('laureate_data.json') as f:
    laureate_data = json.load(f)

### Saving the Json file to local

In [6]:
# # Write prize data to JSON file
# with open('prize_data.json', 'w') as file:
#     json.dump(prize_data, file, indent=4)

# # Write laureate data to JSON file
# with open('laureate_data.json', 'w') as file:
#     json.dump(laureate_data, file, indent=4)

### Creating SQLite Database

In [7]:
# Step 3: Create SQLite database schema
conn = sqlite3.connect('nobel_prizes.db')
cursor = conn.cursor()

### Creating Laureates table

In [8]:
# Step 4: Create database tables

# Create Laureates table
cursor.execute('''CREATE TABLE IF NOT EXISTS Laureates (
               id INTEGER PRIMARY KEY,
               firstname TEXT,
               surname TEXT,
               born DATE,
               died DATE,
               bornCountry TEXT,
               bornCountryCode TEXT,
               bornCity TEXT,
               diedCountry TEXT,
               diedCountryCode TEXT,
               diedCity TEXT,
               gender TEXT,
               affliation_id INTEGER
               )''')

### Inserting Values into Laureates table

In [9]:
for laureate in laureate_data:
    cursor.execute('''INSERT INTO Laureates VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?,?)''', 
                (laureate['id'],
                 laureate.get('firstname', None),
                 laureate.get('surname', None),
                 laureate.get('born', None),
                 laureate.get('died', None),
                 laureate.get('bornCountry', None),
                 laureate.get('bornCountryCode', None),
                 laureate.get('bornCity', None),
                 laureate.get('diedCountry', None),
                 laureate.get('diedCountryCode', None),
                 laureate.get('diedCity', None),
                 laureate.get('gender', None),
                None))
    

### SQL query to view Laureates table

In [10]:
# Execute SQL query to select all rows from Prizes table
cursor.execute('''SELECT * FROM Laureates''')

# Fetch all rows
rows = cursor.fetchall()

# Print the rows
for row in rows:
    print(row)

(1, 'Wilhelm Conrad', 'Röntgen', '1845-03-27', '1923-02-10', 'Prussia (now Germany)', 'DE', 'Lennep (now Remscheid)', 'Germany', 'DE', 'Munich', 'male', None)
(2, 'Hendrik A.', 'Lorentz', '1853-07-18', '1928-02-04', 'the Netherlands', 'NL', 'Arnhem', 'the Netherlands', 'NL', None, 'male', None)
(3, 'Pieter', 'Zeeman', '1865-05-25', '1943-10-09', 'the Netherlands', 'NL', 'Zonnemaire', 'the Netherlands', 'NL', 'Amsterdam', 'male', None)
(4, 'Henri', 'Becquerel', '1852-12-15', '1908-08-25', 'France', 'FR', 'Paris', 'France', 'FR', None, 'male', None)
(5, 'Pierre', 'Curie', '1859-05-15', '1906-04-19', 'France', 'FR', 'Paris', 'France', 'FR', 'Paris', 'male', None)
(6, 'Marie', 'Curie', '1867-11-07', '1934-07-04', 'Russian Empire (now Poland)', 'PL', 'Warsaw', 'France', 'FR', 'Sallanches', 'female', None)
(8, 'Lord', 'Rayleigh', '1842-11-12', '1919-06-30', 'United Kingdom', 'GB', 'Langford Grove, Maldon, Essex', 'United Kingdom', 'GB', None, 'male', None)
(9, 'Philipp', 'Lenard', '1862-06-0

### Creating Affiliations table

In [11]:
# Create Affiliations table
cursor.execute('''CREATE TABLE Affiliations (
               id INTEGER PRIMARY KEY AUTOINCREMENT,
               name TEXT,
               city TEXT,
               country TEXT
               )''')

### Inserting Values into Affiliations table

In [12]:
# Access and print unique affiliations
unique_affiliations = set()

for item in laureate_data:
    for prize in item.get('prizes', []):
        affiliations = prize.get("affiliations", [])
        for affiliation in affiliations:
            if affiliation:  # Check if affiliation is not None
                affiliation_tuple = tuple(affiliation.items())
                unique_affiliations.add(affiliation_tuple)

for affiliation_tuple in unique_affiliations:
    affiliation_dict = dict(affiliation_tuple)
    print(affiliation_dict if affiliation_dict else "None")

{'name': 'Cornell University', 'city': 'Ithaca, NY', 'country': 'USA'}
{'name': 'Norwegian University of Science and Technology (NTNU)', 'city': 'Trondheim', 'country': 'Norway'}
{'name': 'Vanderbilt University School of Medicine', 'city': 'Nashville, TN', 'country': 'USA'}
{'name': 'Lawrence Berkeley National Laboratory', 'city': 'Berkeley, CA', 'country': 'USA'}
{'name': 'Bristol University', 'city': 'Bristol', 'country': 'United Kingdom'}
{'name': 'The Wellcome Trust Sanger Institute', 'city': 'Cambridge', 'country': 'United Kingdom'}
{'name': 'University of Oxford, Royal Society', 'city': 'Oxford', 'country': 'United Kingdom'}
{'name': 'Institute for Chemical Physics of the Academy of Sciences of the USSR', 'city': 'Moscow', 'country': 'USSR (now Russia)'}
{'name': 'Central Research Laboratories, EMI', 'city': 'London', 'country': 'United Kingdom'}
{'name': 'Polarographic Institute of the Czechoslovak Academy of Science', 'city': 'Prague', 'country': 'Czechoslovakia (now Czech Repu

In [13]:
# Access and print affiliations
for item in laureate_data:
    for prize in item.get('prizes', []):
        affiliations = prize.get("affiliations", [])
        for affiliation in affiliations:
            print(affiliation)

{'name': 'Munich University', 'city': 'Munich', 'country': 'Germany'}
{'name': 'Leiden University', 'city': 'Leiden', 'country': 'the Netherlands'}
{'name': 'Amsterdam University', 'city': 'Amsterdam', 'country': 'the Netherlands'}
{'name': 'École Polytechnique', 'city': 'Paris', 'country': 'France'}
{'name': 'École municipale de physique et de chimie industrielles (Municipal School of Industrial Physics and Chemistry)', 'city': 'Paris', 'country': 'France'}
[]
{'name': 'Sorbonne University', 'city': 'Paris', 'country': 'France'}
{'name': 'Royal Institution of Great Britain', 'city': 'London', 'country': 'United Kingdom'}
{'name': 'Kiel University', 'city': 'Kiel', 'country': 'Germany'}
{'name': 'University of Cambridge', 'city': 'Cambridge', 'country': 'United Kingdom'}
{'name': 'University of Chicago', 'city': 'Chicago, IL', 'country': 'USA'}
{'name': 'Sorbonne University', 'city': 'Paris', 'country': 'France'}
{'name': 'Marconi Wireless Telegraph Co. Ltd.', 'city': 'London', 'countr

In [14]:
# Access and print unique affiliations
unique_affiliations = set()

for item in laureate_data:
    for prize in item.get('prizes', []):
        affiliations = prize.get("affiliations", [])
        for affiliation in affiliations:
            if affiliation:  # Check if affiliation is not None
                affiliation_tuple = tuple(affiliation.items())
                unique_affiliations.add(affiliation_tuple)

# Insert unique affiliations into the Affiliations table
for affiliation_tuple in unique_affiliations:
    affiliation_dict = dict(affiliation_tuple)
    if affiliation_dict:
        name = affiliation_dict.get('name', None)
        city = affiliation_dict.get('city', None)
        country = affiliation_dict.get('country', None)
        cursor.execute("INSERT INTO Affiliations (name, city, country) VALUES (?, ?, ?)", (name, city, country))

### SQL query to view Affiliations table

In [15]:
# Execute SQL query to select all rows from Prizes table
cursor.execute('''SELECT * FROM affiliations ORDER BY id ''')

# Fetch all rows
rows = cursor.fetchall()

# Print the rows
for row in rows:
    print(row)

(1, 'Cornell University', 'Ithaca, NY', 'USA')
(2, 'Norwegian University of Science and Technology (NTNU)', 'Trondheim', 'Norway')
(3, 'Vanderbilt University School of Medicine', 'Nashville, TN', 'USA')
(4, 'Lawrence Berkeley National Laboratory', 'Berkeley, CA', 'USA')
(5, 'Bristol University', 'Bristol', 'United Kingdom')
(6, 'The Wellcome Trust Sanger Institute', 'Cambridge', 'United Kingdom')
(7, 'University of Oxford, Royal Society', 'Oxford', 'United Kingdom')
(8, 'Institute for Chemical Physics of the Academy of Sciences of the USSR', 'Moscow', 'USSR (now Russia)')
(9, 'Central Research Laboratories, EMI', 'London', 'United Kingdom')
(10, 'Polarographic Institute of the Czechoslovak Academy of Science', 'Prague', 'Czechoslovakia (now Czech Republic)')
(11, 'Space Telescope Science Institute', 'Baltimore, MD', 'USA')
(12, 'Meijo University', 'Nagoya', 'Japan')
(13, 'Hokkaido University', 'Sapporo', 'Japan')
(14, 'McMaster University', 'Hamilton, Ontario', 'Canada')
(15, "St. Jude

### Creating Prize table

In [16]:
# Create Prizes table
cursor.execute('''CREATE TABLE Prizes (
               id INTEGER PRIMARY KEY AUTOINCREMENT,
               year INTEGER,
               category TEXT,
               share INTEGER,
               motivation TEXT,
               laureate_id1 INTEGER,
               laureate_id2 INTEGER,
               laureate_id3 INTEGER
               )''')

### Inserting Values into Prize table

In [17]:
# Access and print prize and affiliations
for prize in prize_data:
    prize_id = cursor.lastrowid
    year=prize.get('year', None)
    category=prize.get('category', None)
    pairs_to_match = {'year': year,'category':category}
    for item in laureate_data:
        for prize in item.get('prizes', []):
            if all(prize.get(key) == value for key, value in pairs_to_match.items()):
                new_prize=prize
                break
    share=new_prize.get('share',None)
    motivation=new_prize.get('motivation','None')
    print(year,category,share,motivation)
    cursor.execute("INSERT INTO Prizes (year, category, share,motivation) VALUES (?, ?, ?,?)", (year, category, share,motivation))

2023 chemistry 3 "for the discovery and synthesis of quantum dots"
2023 economics 1 "for having advanced our understanding of women’s labour market outcomes"
2023 literature 1 "for his innovative plays and prose which give voice to the unsayable"
2023 peace 1 "for her fight against the oppression of women in Iran and her fight to promote human rights and freedom for all"
2023 physics 3 "for experimental methods that generate attosecond pulses of light for the study of electron dynamics in matter"
2023 medicine 2 "for their discoveries concerning nucleoside base modifications that enabled the development of effective mRNA vaccines against COVID-19"
2022 chemistry 3 "for the development of click chemistry and bioorthogonal chemistry"
2022 economics 3 "for research on banks and financial crises"
2022 literature 1 "for the courage and clinical acuity with which she uncovers the roots, estrangements and  collective restraints of personal memory"
2022 peace 3 "The Peace Prize laureates repre

1986 chemistry 3 "for their contributions concerning the dynamics of chemical elementary processes"
1986 economics 1 "for his development of the contractual and constitutional bases for the theory of economic and political decision-making"
1986 literature 1 "who in a wide cultural perspective and with poetic overtones fashions the drama of existence"
1986 peace 1 "for being a messenger to mankind: his message is one of peace, atonement and dignity"
1986 physics 4 "for their design of the scanning tunneling microscope"
1986 medicine 2 "for their discoveries of growth factors"
1985 chemistry 2 "for their outstanding achievements in the development of direct methods for the determination of crystal structures"
1985 economics 1 "for his pioneering analyses of saving and of financial markets"
1985 literature 1 "who in his novel combines the poet's and the painter's creativeness with a deepened awareness of time in the depiction of the human condition"
1985 peace 1 "for spreading authoritati

### SQL query to view Prizes table

In [18]:
# Execute SQL query to select all rows from Prizes table
cursor.execute('''SELECT * FROM Prizes''')

# Fetch all rows
rows = cursor.fetchall()

# Print the rows
for row in rows:
    print(row)

(1, 2023, 'chemistry', 3, '"for the discovery and synthesis of quantum dots"', None, None, None)
(2, 2023, 'economics', 1, '"for having advanced our understanding of women’s labour market outcomes"', None, None, None)
(3, 2023, 'literature', 1, '"for his innovative plays and prose which give voice to the unsayable"', None, None, None)
(4, 2023, 'peace', 1, '"for her fight against the oppression of women in Iran and her fight to promote human rights and freedom for all"', None, None, None)
(5, 2023, 'physics', 3, '"for experimental methods that generate attosecond pulses of light for the study of electron dynamics in matter"', None, None, None)
(6, 2023, 'medicine', 2, '"for their discoveries concerning nucleoside base modifications that enabled the development of effective mRNA vaccines against COVID-19"', None, None, None)
(7, 2022, 'chemistry', 3, '"for the development of click chemistry and bioorthogonal chemistry"', None, None, None)
(8, 2022, 'economics', 3, '"for research on bank

In [19]:
for item in laureate_data:
    item_id=item.get('id')
    for prize in item.get('prizes', []):
        affiliations = prize.get("affiliations", [])
        for affiliation in affiliations:
            if affiliation:
                name=affiliation.get('name',None)
                country=affiliation.get('country',None)
                city=affiliation.get('city',None)
    cursor.execute('''SELECT id FROM affiliations where name=(?) and country=(?) and city=(?) ''',(name,country,city))
    rows = cursor.fetchall()
    if rows:
        aff_id=rows[0][0]
    print(aff_id)
    cursor.execute('''UPDATE Laureates SET affliation_id=? WHERE id=?''', (aff_id,item_id))

219
243
36
341
367
205
168
359
112
147
205
100
290
36
332
232
243
266
167
106
253
279
252
149
96
97
107
353
213
92
205
147
112
177
35
352
145
279
112
190
308
107
227
177
343
267
282
261
319
309
46
106
260
5
26
209
16
309
165
253
56
16
261
316
28
364
196
261
224
224
267
267
267
16
107
238
319
256
313
126
224
224
93
309
107
329
1
267
107
199
321
166
25
275
124
317
112
112
112
293
40
261
119
126
364
112
309
238
197
197
226
166
309
147
319
309
16
353
1
147
107
244
244
153
163
158
158
158
158
270
182
244
309
89
189
126
126
16
231
244
319
319
14
126
16
175
1
16
1
16
329
142
16
261
319
120
120
279
279
369
167
219
205
306
106
145
213
44
173
246
309
133
95
279
82
112
274
213
353
219
213
177
369
265
294
75
317
261
138
138
96
283
246
313
279
335
369
198
113
1
342
342
82
353
267
359
55
267
267
178
264
239
107
1
82
161
112
181
10
141
267
181
181
66
179
7
309
147
272
193
168
235
166
273
284
298
334
78
78
276
166
16
215
335
309
65
32
324
313
16
203
260
1
181
16
78
110
47
309
267
154
141
231
18
18
143

In [20]:
for item in prize_data:
    year=item.get('year',None)
    category=item.get('category',None)
    laureates=item.get('laureates',[])
    if(len(laureates)==3):
        cursor.execute('''UPDATE Prizes SET laureate_id1=?,laureate_id2=?,laureate_id3=? WHERE year=? and category=?''', (laureates[0].get('id',None),laureates[1].get('id',None),laureates[2].get('id',None),year,category))
    elif(len(laureates)==2):
        cursor.execute('''UPDATE Prizes SET laureate_id1=?,laureate_id2=? WHERE year=? and category=?''', (laureates[0].get('id',None),laureates[1].get('id',None),year,category))
    elif(len(laureates)==1):
        cursor.execute('''UPDATE Prizes SET laureate_id1=? WHERE year=? and category=?''', (laureates[0].get('id',None),year,category))
    

In [ ]:
# Step 6: Commit the transaction and close the database connection
# conn.commit()
# conn.close()